# Create a Marquee Portfolio with GS Quant

The Marquee Portfolio Service provides a powerful framework for uploading portfolio positions and retrieving analytics including historical performance, factor risk exposure, ESG analytics, and more. GS Quant makes operating the suite of Portfolio Service API's intuitive and fast.

## Permission Prerequisites

To execute all the code in this tutorial, you will need the following application scopes:
- **read_product_data**
- **read_financial_data**
- **modify_product_data** (must be requested)
- **modify_financial_data** (must be requested)
- **run_analytics** (must be requested)
- **read_user_profile**

If you are not yet permissioned for these scopes, please request them on your [My Applications Page](https://developer.gs.com/go/apps/view). If you have any other questions please reach out to the [Marquee sales team](mailto:gs-marquee-sales@gs.com).


## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import datetime as dt

from gs_quant.entities.entitlements import Entitlements, EntitlementBlock, User
from gs_quant.markets.portfolio import Portfolio
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.markets.report import CustomAUMDataPoint
from gs_quant.markets.position_set import Position, PositionSet
from gs_quant.session import GsSession, Environment
from gs_quant.target.portfolios import RiskAumSource

client = 'ENTER CLIENT ID'
secret = 'ENTER CLIENT SECRET'

GsSession.use(
    Environment.PROD,
    client_id=client,
    client_secret=secret,
    scopes=('read_product_data read_financial_data modify_product_data modify_financial_data run_analytics read_user_profile',)
)

print('GS Session initialized.')

## Step 2: Create the Portfolio

The first step is to create a new, empty portfolio in Marquee.

In [ ]:
portfolio = Portfolio(name='My New Portfolio')
portfolio.save()

print(f"Created portfolio '{portfolio.name}' with ID: {portfolio.id}")

Once your portfolio has been saved to Marquee, the `PortfolioManager` class allows users to interact with their Marquee portfolios directly from GS Quant. We will be using `PortfolioManager` to update portfolio positions, entitlements, update custom AUM, and run reports.

## Step 3: Define Portfolio Entitlements

By default, an application will have all entitlement permissions to a portfolio it makes. However, if you would like to share the portfolio with others, either Marquee users or other applications, you will need to specify them in the entitlements parameter of the portfolio. Let's walk through how we convert a list of admin and viewer emails into an `Entitlements` object:

In [ ]:
portfolio_admin_emails = ['LIST OF ADMIN EMAILS']
portfolio_viewer_emails = ['LIST OF VIEWER EMAILS']

admin_entitlements = EntitlementBlock(users=User.get_many(emails=portfolio_admin_emails))
view_entitlements = EntitlementBlock(users=User.get_many(emails=portfolio_viewer_emails))

entitlements = Entitlements(
    view=view_entitlements,
    admin=admin_entitlements
)

print(f'Entitlements:\n{entitlements.to_dict()}')

pm = PortfolioManager(portfolio.id)
pm.set_entitlements(entitlements)

print(f"Updated entitlements for '{portfolio.name}'")

## Step 4: Define Portfolio Positions

Portfolio positions in Marquee are stored on a holding basis, when means you only upload positions for days where you are rebalancing your portfolio. Take the following set of positions:

In [ ]:
portfolio_position_sets = [
    PositionSet(
        date=dt.date(day=3, month=5, year=2021),
        positions=[
            Position(identifier='AAPL UW', quantity=25),
            Position(identifier='GS UN', quantity=50)
        ]
    ),
    PositionSet(
        date=dt.date(day=1, month=7, year=2021),
        positions=[
            Position(identifier='AAPL UW', quantity=26),
            Position(identifier='GS UN', quantity=51)
        ]
    )
]

If these positions were to be uploaded correctly, this portfolio would hold 50 shares of GS UN and 25 shares of AAPL UW from May 3, 2021 to June 30, 2021, and it would hold 51 shares of GS UN and 26 shares of AAPL UW from July 1, 2021 to today.

In [ ]:
pm.update_positions(portfolio_position_sets)

print(f"Updated positions for '{portfolio.name}'")

## Step 5: Schedule Reports
By default, creating a portfolio will automatically create a corresponding Performance Report for it as well. If you would like to create a Factor Risk Report for it as well, follow the steps [here](../examples/marquee/00_create_factor_risk_report.ipynb). Then, remember to schedule all the portfolio reports.

In [ ]:
pm.schedule_reports()

print('All portfolio reports scheduled.')

## Step 6: Update Custom AUM (Optional)
The `CustomAUMDataPoint` class is used to represent custom AUM data for a specific date. A list of them can be posted to Marquee using our initialized `PortfolioManager`. If you do not upload custom AUM data for your portfolio and change your portfolio's AUM Source to `Custom AUM`, by default the "AUM" (which is used for calculating risk as percent values) will be your portfolio's long exposure.

In [ ]:
performance_report = pm.get_performance_report()
performance_report.set_aum_source(RiskAumSource.Custom_AUM)
custom_aum = [
    CustomAUMDataPoint(date=dt.date(2021, 5, 1), aum=100000),
    CustomAUMDataPoint(date=dt.date(2021, 7, 1), aum=200000)
]
performance_report.upload_custom_aum(custom_aum, clear_existing_data=False)

print(f"Custom AUM for '{portfolio.name} successfully uploaded'")

### You're all set, Congrats! What's next?

* [Creating and scheduling a factor risk report](../examples/marquee/00_create_factor_risk_report.ipynb)

* [Updating the portfolio with new positions](../tutorials/Update%20Historical%20Portfolio.ipynb)

* [Retrieving the portfolio's performance analytics](../tutorials/Pull%20Portfolio%20Performance%20Data.ipynb)

* [Retrieving the portfolio's factor risk and attribution analytics](../tutorials/Pull%20Portfolio%20Factor%20Risk%20Data.ipynb)


*Other questions? Reach out to the [Portfolio Analytics team](mailto:gs-marquee-analytics-support@gs.com)!*
